# Probability

## Pro Tips
- Important concepts to review from an introductory probability class include the [Law of Total Probability](#law_of_total_probability), [Bayes' Rule](#Bayes), and Expectation. You can learn many of these topics (and important topics regarding hypothesis testing and inference) with intro-level courses in probability and inference.

In [1]:
from __future__ import division
import numpy as np
from collections import Counter
import random
import scipy.stats

### 1. Bobo the amoeba has a 25%, 25%, and 50%  chance of producing 0, 1, or 2 offspring, respecitvely. Each of Bobo's descendants also have the same probabilities. What is the probability that Bobo’s lineage dies out?

We are given:<br>
$ P[X_n = 0] = .25 $<br>
$ P[X_n = 1] = .25 $<br>
$ P[X_n = 2] = .5 $<br>
Note that we specify $ X_n $ for all probabilities, not just $ X $ because the probabilities are the same for every generation.

To find the probability $ q $ that Bobo's lineage dies out, we can look at each possible scenario:<br>
- If Bobo has no offspring, the lineage has died out.<br>
$ P[X_1 = 0] = .25 $<br>
- If Bobo has one offspring, that offspring has the same probability of dying out as Bobo.<br>
$ P[X_1 = 1 \cap X_2 = 0] = .25q $
- If Bobo has two offspring, we want to find the probability that both offspring die out.<br>
$ P[X_1 = 2 \cap X_2 = 0] = .5q^2 $

So, we have $ q = .25 + .25q + .5q^2 $. Solving for the quadratic, we find that $ q = .5 $

In [2]:
## Code to simulate the scenario:

def generation(n=1): # n is number of living amoebas
    if n > 100:
        return True # consider survived
    else:
        next_gen = 0
        # for each amoeba in current generation count descendants
        for amoeba in xrange(n):
            # random number of descendants: 0-3
            descendants = np.random.randint(0,4)
            # 3 descendants are only two :)
            if descendants == 3:
                descendants = 2
            next_gen += descendants
        if next_gen == 0:
            return False # died out
        else:
            return generation(n=next_gen)

experiments = 10000
survived = 0
for experiment in xrange(experiments):
    if generation():
        survived += 1
print 'survival rate: {}% (in {} experiments)'.format(100*survived/experiments, experiments)

survival rate: 51.15% (in 10000 experiments)


### 2. In any 15-minute interval, there is a 20% probability that you will see at least one shooting star. What is the probability that you see at least one shooting star in the period of an hour?

We are given:<br>
$ P[X \geq 1] = .2 $ in a 15-minute interval.<br>
By the [Law of Total Probability](#law_of_total_probability), we know that $ P[X = 0] = .8 $.

We want to find $ 1 - P[X = 0]^4 $ since there are 4 15-minute intervals in an hour.<br>
$ 1 - P[X = 0]^4 = 1 - 0.8^4 = .5904 $

In [3]:
## Code to simulate the scenario:

def hour():
    for quarter in xrange(4):
        if np.random.randint(0,5) == 0:
            return True
    return False

experiments = 10000
stars = 0
for experiment in xrange(experiments):
    if hour():
        stars += 1
print 'saw at least one star: {}% (in {} experiments)'.format(100*stars/experiments, experiments)

saw at least one star: 58.92% (in 10000 experiments)


### 3. How can you generate a random number between 1 - 7 with only a die?

Roll the die twice and add the outcomes of the rolls to get 36 possible outcomes with uniform probability. Assign 1 outcome (e.g., two sixes) as a "dummy" outcome and divide the remaining 35 outcomes into 7 "buckets", each assigned to a number from 1 to 7. If you roll the dummy outcome, roll again.

In [4]:
## See if it works (it is not ideal but I am going to blame numpy non-randomness):

x = np.random.randint(1, 7, size=10000)
y = np.random.randint(1, 7, size=10000)
print Counter((6*(x-1) + y) % 7 + 1)

Counter({2: 1676, 4: 1421, 7: 1407, 5: 1400, 6: 1387, 1: 1384, 3: 1325})


### 4. How can you get a fair coin toss if someone hands you a coin that is weighted to come up heads more often than tails?

Flip the coin twice and assign a heads-then-tails event as a "Heads" outcome and a tails-then-heads event as a "Tails" outcome, or vice versa. The probability of getting a head first, then a tail is the same probability as flipping a tail first, then a head. If you flip two heads in a row or two tails in a row, try again.

Proof:<br>
Let $ p $ = the probability of flipping heads, then the probability of flipping tails is $ 1 - p $.<br>
$ P[X_1 = H \cap X_2 = T] = p(1 - p) = (1 - p)p = P[X_1 = T \cap X_2 = H] $

In [5]:
## Code to simulate the scenario:

def flip(p):
    return 'H' if random.random() < p else 'T'

experiments = 10000
results = []
p = 0.8
for experiment in xrange(experiments):
    x = flip(p)
    y = flip(p)
    if x != y:
        results.append(x)

print Counter(results)

Counter({'T': 1643, 'H': 1544})


### 5. You have an 50-50 mixture of two normal distributions with the same standard deviation. How far apart do the means need to be in order for this distribution to be bimodal?

Definition: Bimodal distribution is a continuous probability distribution with two different modes. These appear as distinct peaks (local maxima) in the probability density function.

The probability of $ x $ in the mixture is given by $ p(x) = 0.5N(x; \mu_1, \sigma) + 0.5N(x; \mu_2, \sigma) $ which is symmetric around $ \frac{\mu_1 + \mu_2}{2} $. The mixture is bimodal if and only if $ \mid \mu_1 − \mu_2 \mid > 2\sigma $.

### 6. Given draws from a normal distribution with known parameters, how can you simulate draws from a uniform distribution?

There is actually a technique called Probability Integral Transform (aka Universality of the Uniform) which says "If you plug in a random variable into its own CDF, you get a Uniform distribution." So just plug your draw into the CDF of the normal distribution.

Not really a proof but this is the idea in code:

In [6]:
def draw():
    draw = scipy.stats.norm.rvs()
    return int(scipy.stats.norm.cdf(draw)*10) #multiply by 10 and round down to int

results = []
experiments = 10000
for experiment in xrange(experiments):
    results.append(draw())

print Counter(results)

Counter({2: 1073, 7: 1038, 9: 1028, 5: 1014, 8: 1011, 6: 983, 1: 979, 3: 968, 4: 965, 0: 941})


### 7. A certain couple tells you that they have two children, at least one of which is a girl. What is the probability that they have two girls?

Assuming the chances for boy vs girl are equal, the sample space is {BB, BG, GB, GG}.
We know their case is not BB, which leaves us with BG, GB, GG. The chance of two girls is 1/3.

Some experiment code:

In [7]:
x = np.random.randint(0, 2, size=10000) # 1st child (girl 0, boy 1)
y = np.random.randint(0, 2, size=10000) # 2nd child (girl 0, boy 1)

# two girls: x + y = 0
# at least one girl: x + y <=1
print sum(x + y == 0) / sum(x + y <= 1)

0.330343996788


### 8. You have a group of couples that decide to have children until they have their first girl, after which they stop having children. What is the expected gender ratio of the children that are born? What is the expected number of children each couple will have?

Intuitively: All couples have their first child and those children are 50/50 boys and girls. Half of the couples (those with boys) has another child and those children are again 50/50 boys and girls. This continues but the ration remains and there is the same number of girls and boys. Expected number of children for each couple is 2.

In [8]:
def generation(n=10, g=0, b=0): # n is number of couples
    if n == 0:
        return g, b
    else:
        children = np.random.randint(0, 2, size=n) # girl 0, boy 1
        g += sum(children == 0)
        b += sum(children == 1)
        return generation(n=sum(children == 1), g=g, b=b)

couples = 10000
girls, boys = generation(n=couples)
print '{} girls, {} boys, {} children per couple'.format(girls, boys, (girls+boys)/couples)

10000 girls, 9950 boys, 1.995 children per couple


### 9. How many ways can you split 12 people into 3 teams of 4?

$\begin{pmatrix} 12 \\ 4 \end{pmatrix}$ for the first group = 495<br>
$\begin{pmatrix} 8 \\ 4 \end{pmatrix}$ for the second group = 70<br>
$\begin{pmatrix} 4 \\ 4 \end{pmatrix}$ for the third group = 1<br>
multiply those numbers and divide by $ 3! $ because we do not care about the order of the groups

result: $ \frac{495 * 70 * 1}{3!} = 5775 $

### 10. Your hash function assigns each object to a number between 1:10, each with equal probability. With 10 objects, what is the probability of a hash collision? What is the expected number of hash collisions? What is the expected number of hashes that are unused.

### 11. You call 2 UberX's and 3 Lyfts. If the time that each takes to reach you is IID, what is the probability that all the Lyfts arrive first? What is the probability that all the UberX's arrive first?

### 12. I write a program should print out all the numbers from 1 to 300, but prints out 'Fizz' instead if the number is divisible by 3, 'Buzz' instead if the number is divisible by 5, and 'FizzBuzz' if the number is divisible by 3 and 5. What is the total number of numbers that is either 'Fizz'ed, 'Buzz'ed, or 'FizzBuzz'ed?

### 13. On a dating site, users can select 5 out of 24 adjectives to describe themselves. A match is declared between two users if they match on at least 4 adjectives. If Alice and Bob randomly pick adjectives, what is the probability that they form a match?

### 14. A lazy high school senior types up application and envelopes to `n` different colleges, but puts the applications randomly into the envelopes. What is the expected number of applications that went to the right college?

### 15. Let’s say you have a very tall father. On average, what would you expect the height of his son to be? Taller, equal, or shorter? What if you had a very short father?

### 16. What's the expected number of coin flips until you get two heads in a row? What’s the expected number of coin flips until you get two tails in a row?

### 17. Let's say we play a game where I keep flipping a coin until I get head. If the first time I get heads is on the `n`th coin, then I pay you 2`n`-1 dollars. How much would you pay me to play this game?

<a id='18'></a>
### 18. You have two coins, one of which is fair and comes up heads with a probability 1/2, and the other which is biased and comes up heads with probaility 3/4. You randomly pick coin and flip it twice, and get heads both times. What is the probability that you picked the fair coin?

### 19. You have a 0.1% chance of picking up a coin with both heads, and a 99.9% chance that you pick up a fair coin. You flip your coin and it comes up heads 10 times. What’s the chance that you picked up the fair coin, given the information that you observed?

<a id='law_of_total_probability'></a>
### [Law of Total Probability](https://en.wikipedia.org/wiki/Law_of_total_probability)

The total probability of some event $ A $ is equal to the sum of disjoint $ \{B: 1, 2, 3, ... n\} $ where $ A $ and the union of all $ B_n $ occupy the same sample space $ S $ in entirety.

<img src='images/law_of_total_probability.png' width=200>

In math words:
$$ P[A] = \sum_n P[A \mid B_n]P[B_n] = \sum_n P[A \cap B_n] $$

The expanded version for conditional probability:
$$ P[A \mid C] = \sum_n P[A \mid C \cap B_n]P[B_n] = \sum_n P[A \cap B_n \mid C] $$

Image stolen from [here](https://www.probabilitycourse.com/chapter1/1_4_2_total_probability.php).

<a id='Bayes'></a>
### [Bayes' Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem) (more commonly known as Bayes' Theorem)

$$ P[A \mid B] = \frac{P[B \mid A]P[A]}{P[B]} = \frac{P[A \cap B]}{P[B]} $$

**Pro Tip:**<br>
Many Bayes' Rule questions can be solved quickly with the odds form of Bayes' Rule, which says that prior odds times likelihood ratio is the posterior odds. For problem [18](#18), the prior odds is 999:1 and the likelihood ratio is 1/1024:1 (10 heads has a 1/1024 probability with a fair coin and 1 probability with a biased coin), which means the posterior odds is about 1:1. For problem 19, the prior odds is 1:1 and the likelihood ratio is 1/4:9/16, so the posterior odds is 4:9.